### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Stroke/GSE186798'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
# Given the dataset involves microarrays, it is likely to contain gene expression data
is_gene_available = True

# Determine the availability of variables 'Stroke', 'age', and 'gender'
# Gender data is available in the sample characteristics dictionary (key 0)
# Condition (which could indicate 'Stroke') is available in the sample characteristics dictionary (key 1)

gender_row = 0
trait_row = 1

# Age data is not explicitly mentioned in the provided information, so we set age_row to None

# Define conversion functions
def convert_trait(value):
    value = value.split(':')[1].strip().upper()
    if value == 'PSD':
        return 1
    elif value == 'PSND' or value == 'CONTROL':
        return 0
    else:
        return None

def convert_age(value):
    return None  # Since age data is not available

def convert_gender(value):
    value = value.split(':')[1].strip().upper()
    if value == 'F':
        return 0
    elif value == 'M':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE186798', './preprocessed/Stroke/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Stroke', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Stroke/trait_data/GSE186798.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:

# 1. Identify the keys for identifier and gene symbol
identifier_key = 'probeset_id'
gene_symbol_key = 'mrna_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 2.1 Parse the gene symbols from the complex 'mrna_assignment' data
mapping_df['Gene'] = mapping_df[gene_symbol_key].apply(lambda x: x.split('//')[1].strip() if '//' in x else None)

# 3. Remove unnecessary columns and keep only 'ID' and 'Gene'
mapping_df = mapping_df[[identifier_key, 'Gene']]
mapping_df = mapping_df.rename(columns={identifier_key: 'ID'})

# 4. Apply the mapping with the 'apply_gene_mapping' function from the library and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Stroke/gene_data/GSE186798.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Stroke')

# 4. Save the cohort information.
save_cohort_info('GSE186798', './preprocessed/Stroke/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Stroke/GSE186798.csv'
    unbiased_merged_data.to_csv(csv_path)
